In [92]:
import numpy as np

# -------------------------------------------------------------
# 1. Representasi embedding sederhana dari 8 token
# -------------------------------------------------------------
X = np.array([
    [ 1,  0,  1,  0, -1,  0],   # gimana
    [ 0,  1,  0, -1,  1,  0],   # nih
    [ 1, -1,  0,  0,  0,  1],   # apa
    [ 0,  0, -1,  1,  0,  1],   # gak
    [-1,  1,  0,  1,  0,  0],   # ken
    [ 1,  0, -1,  0,  1,  0],   # update
    [ 0,  1,  1,  0,  0, -1],   # kai
    [ 1,  0,  0,  1, -1,  0]    # access
])

# -------------------------------------------------------------
# 2. Matriks bobot proyeksi 0Query, Key, dan Value
#    (6 dimensi → 3 dimensi)
# -------------------------------------------------------------
WQ = np.array([
    [ 1,  0,  1],
    [ 0,  1, -1],
    [ 1, -1,  0],
    [ 0,  1,  0],
    [-1,  0,  1],
    [ 1,  0, -1]
])

WK = np.array([
    [ 0,  1,  0],
    [ 1, -1,  0],
    [-1,  0,  1],
    [ 1,  0, -1],
    [ 0,  1,  1],
    [ 1,  0,  0]
])

WV = np.array([
    [ 1,  0,  0],
    [ 0,  1,  0],
    [ 1,  1, -1],
    [ 0,  0,  1],
    [ 1,  0,  1],
    [ 0, -1,  1]
])

# -------------------------------------------------------------
# 3. Menghitung Q, K, V
# -------------------------------------------------------------

In [93]:
Q = X @ WQ
Q

array([[ 3, -1,  0],
       [-1,  0,  0],
       [ 2, -1,  1],
       [ 0,  2, -1],
       [-1,  2, -2],
       [-1,  1,  2],
       [ 0,  0,  0],
       [ 2,  1,  0]])

In [94]:
K = X @ WK
K

array([[-1,  0,  0],
       [ 0,  0,  2],
       [ 0,  2,  0],
       [ 3,  0, -2],
       [ 2, -2, -1],
       [ 1,  2,  0],
       [-1, -1,  1],
       [ 1,  0, -2]])

In [95]:
V = X @ WV
V

array([[ 1,  1, -2],
       [ 1,  1,  0],
       [ 1, -2,  1],
       [-1, -2,  3],
       [-1,  1,  1],
       [ 1, -1,  2],
       [ 1,  3, -2],
       [ 0,  0,  0]])

In [96]:
K.T

array([[-1,  0,  0,  3,  2,  1, -1,  1],
       [ 0,  0,  2,  0, -2,  2, -1,  0],
       [ 0,  2,  0, -2, -1,  0,  1, -2]])

In [97]:
# Qkt = Q @ K.T
# Qkt
# -------------------------------------------------------------
# 4. Attention Score (QKᵀ)
# -------------------------------------------------------------
attention_scores1 = Q @ K.T
attention_scores1

array([[-3,  0, -2,  9,  8,  1, -2,  3],
       [ 1,  0,  0, -3, -2, -1,  1, -1],
       [-2,  2, -2,  4,  5,  0,  0,  0],
       [ 0, -2,  4,  2, -3,  4, -3,  2],
       [ 1, -4,  4,  1, -4,  3, -3,  3],
       [ 1,  4,  2, -7, -6,  1,  2, -5],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [-2,  0,  2,  6,  2,  4, -3,  2]])

In [98]:
# # -------------------------------------------------------------
# # 5. Softmax untuk bobot perhatian
# # -------------------------------------------------------------
# # def softmax(x):
# #     e = np.exp(x - np.max(x))
# #     return e / e.sum(axis=-1, keepdims=True)
# def softmax(x):
#     # stabilitas numerik: kurangi nilai maksimum di tiap baris
#     e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
#     return e_x / np.sum(e_x, axis=1, keepdims=True)

# attention_weights1 = softmax(attention_scores1)

# # -------------------------------------------------------------
# # 6. Attention Output
# # -------------------------------------------------------------
# attention_output1 = attention_weights1 @ V

# print("\nAttention Scores:\n", attention_scores1)
# print("\nAttention Weights:\n", attention_weights1)
# print("\nContextual Embedding (Attention Output):\n", attention_output1)


In [99]:
d_k = 3
sqrt_dk = np.sqrt(d_k)
scaled_attention_logits = attention_scores1 / sqrt_dk
scaled_attention_logits

array([[-1.73205081,  0.        , -1.15470054,  5.19615242,  4.61880215,
         0.57735027, -1.15470054,  1.73205081],
       [ 0.57735027,  0.        ,  0.        , -1.73205081, -1.15470054,
        -0.57735027,  0.57735027, -0.57735027],
       [-1.15470054,  1.15470054, -1.15470054,  2.30940108,  2.88675135,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -1.15470054,  2.30940108,  1.15470054, -1.73205081,
         2.30940108, -1.73205081,  1.15470054],
       [ 0.57735027, -2.30940108,  2.30940108,  0.57735027, -2.30940108,
         1.73205081, -1.73205081,  1.73205081],
       [ 0.57735027,  2.30940108,  1.15470054, -4.04145188, -3.46410162,
         0.57735027,  1.15470054, -2.88675135],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-1.15470054,  0.        ,  1.15470054,  3.46410162,  1.15470054,
         2.30940108, -1.73205081,  1.15470054]])

In [100]:
def softmax(x):
    # stabilitas numerik: kurangi nilai maksimum di tiap baris
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

# def softmax1(x):
#     e = np.exp(x - np.max(x))
#     return e / e.sum(axis=-1, keepdims=True)

attention_weights2 = softmax(scaled_attention_logits)
attention_weights2

array([[6.07581153e-04, 3.43419065e-03, 1.08229171e-03, 6.20132834e-01,
        3.48132597e-01, 6.11736562e-03, 1.08229171e-03, 1.94108481e-02],
       [2.48181232e-01, 1.39324951e-01, 1.39324951e-01, 2.46495384e-02,
        4.39085229e-02, 7.82147864e-02, 2.48181232e-01, 7.82147864e-02],
       [9.05434064e-03, 9.11626568e-02, 9.05434064e-03, 2.89265770e-01,
        5.15272638e-01, 2.87300844e-02, 2.87300844e-02, 2.87300844e-02],
       [3.55215487e-02, 1.11946835e-02, 3.57644900e-01, 1.12712469e-01,
        6.28451524e-03, 3.57644900e-01, 6.28451524e-03, 1.12712469e-01],
       [7.03768664e-02, 3.92400675e-03, 3.97786494e-01, 7.03768664e-02,
        3.92400675e-03, 2.23310939e-01, 6.98988099e-03, 2.23310939e-01],
       [8.87028441e-02, 5.01369202e-01, 1.58007456e-01, 8.75018543e-04,
        1.55868118e-03, 8.87028441e-02, 1.58007456e-01, 2.77649777e-03],
       [1.25000000e-01, 1.25000000e-01, 1.25000000e-01, 1.25000000e-01,
        1.25000000e-01, 1.25000000e-01, 1.25000000e-01, 1.

In [101]:
np.set_printoptions(formatter={'float': lambda x: f"{x:0.3f}"})

print(attention_weights2)

[[0.001 0.003 0.001 0.620 0.348 0.006 0.001 0.019]
 [0.248 0.139 0.139 0.025 0.044 0.078 0.248 0.078]
 [0.009 0.091 0.009 0.289 0.515 0.029 0.029 0.029]
 [0.036 0.011 0.358 0.113 0.006 0.358 0.006 0.113]
 [0.070 0.004 0.398 0.070 0.004 0.223 0.007 0.223]
 [0.089 0.501 0.158 0.001 0.002 0.089 0.158 0.003]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.006 0.019 0.060 0.602 0.060 0.190 0.003 0.060]]


In [102]:
attention_output = attention_weights2 @ V
attention_output

array([[-0.95594171, -0.89312637,  2.21846838],
       [ 0.78466909,  0.76979464, -0.57911326],
       [-0.6378069 ,  0.07630958,  1.37401561],
       [ 0.64929356, -1.22650534,  1.33374449],
       [ 0.62808731, -1.06044314,  0.90472948],
       [ 0.9923561 ,  0.6591853 , -0.15382372],
       [ 0.375     ,  0.125     ,  0.375     ],
       [-0.38446667, -1.41985083,  2.28828779]])

In [103]:
np.random.seed(42)
wq2, wk2, wv2 = [np.random.randint(-1, 2, (9, 3)) for _ in range(3)]
wq3, wk3, wv3 = [np.random.randint(-1, 2, (9, 3)) for _ in range(3)]

In [104]:
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def attention(X, Wq, Wk, Wv):
    d_k = Wq.shape[1]
    Q, K, V = X @ Wq, X @ Wk, X @ Wv
    scores = (Q @ K.T) / np.sqrt(d_k)
    weights = softmax(scores)
    return weights @ V

In [105]:
head2 = attention(X, wq2, wk2, wv2)
head3 = attention(X, wq3, wk3, wv3)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 6)

In [18]:
head2

array([[0.318, -0.560, -1.352],
       [-0.404, -0.818, -0.823],
       [-0.966, -1.932, -0.969],
       [0.009, -0.984, -0.997],
       [0.019, 0.851, -1.920],
       [-0.111, 0.171, -0.968],
       [0.480, -0.484, -1.483],
       [-0.001, 0.897, -1.815]])

In [19]:
head3

array([[-2.716, 2.349, 1.096],
       [-2.860, 1.814, 0.886],
       [-3.632, 2.502, 1.549],
       [-1.412, 2.551, 1.983],
       [-1.058, 2.012, 0.035],
       [-1.731, 1.885, -0.321],
       [-1.441, 2.192, 0.572],
       [-3.571, 0.524, -2.125]])

In [20]:
multi_head_concat = np.concatenate([attention_output, head2, head3], axis=1)
multi_head_concat

array([[1.167, -0.426, 1.981, 0.318, -0.560, -1.352, -2.716, 2.349,
        1.096],
       [2.249, -0.971, 1.100, -0.404, -0.818, -0.823, -2.860, 1.814,
        0.886],
       [2.249, -0.971, 1.100, -0.966, -1.932, -0.969, -3.632, 2.502,
        1.549],
       [-0.796, -1.880, 5.464, 0.009, -0.984, -0.997, -1.412, 2.551,
        1.983],
       [-0.085, -0.981, 2.796, 0.019, 0.851, -1.920, -1.058, 2.012,
        0.035],
       [2.365, -1.072, 1.335, -0.111, 0.171, -0.968, -1.731, 1.885,
        -0.321],
       [1.000, -1.000, 1.206, 0.480, -0.484, -1.483, -1.441, 2.192,
        0.572],
       [0.579, -1.232, 2.166, -0.001, 0.897, -1.815, -3.571, 0.524,
        -2.125]])

In [21]:
W_o = np.random.randint(-1, 2, (9, 9))
W_o

array([[ 0,  0,  0, -1,  0, -1, -1,  0,  0],
       [ 0,  1,  0,  1, -1,  1,  0, -1, -1],
       [-1,  1,  0, -1, -1, -1,  1,  1,  0],
       [ 1, -1,  0, -1, -1,  1,  0,  1,  1],
       [ 0, -1, -1,  0, -1,  0,  0,  1,  0],
       [ 1, -1, -1, -1, -1,  1, -1,  0,  0],
       [ 0,  1, -1, -1, -1,  1,  0,  0, -1],
       [ 0,  0,  1,  1,  1,  1, -1,  1,  0],
       [-1,  0,  0,  0,  1,  1, -1, -1,  1]])

In [22]:
attention_output_final = multi_head_concat @ W_o
attention_output_final

array([[-4.110, 0.433, 6.976, 2.525, 6.198, -3.877, -1.279, 3.419, 4.555],
       [-3.214, -0.685, 6.315, 1.581, 7.476, -5.708, -3.026, 1.778,
        4.312],
       [-4.585, 0.365, 9.035, 3.748, 11.421, -5.836, -4.231, 0.126,
        5.186],
       [-8.436, 4.145, 5.943, -1.598, 4.334, -4.414, 2.723, 6.935, 5.283],
       [-4.732, 1.807, 4.138, 1.279, 2.339, -4.605, 2.754, 6.623, 2.092],
       [-2.094, -0.559, 4.414, -0.076, 3.941, -6.018, -1.626, 4.672,
        2.371],
       [-2.782, 0.252, 5.600, 1.431, 5.486, -2.888, -1.074, 3.822, 3.493],
       [-1.857, -1.718, 5.012, 1.934, 1.954, -10.964, 5.003, 6.943,
        2.677]])

In [23]:
def xavier_uniform(shape, rng=np.random):
    fan_in, fan_out = shape[0], shape[1]
    limit = np.sqrt(6.0/(fan_in + fan_out))
    return rng.uniform(-limit, limit, size=shape)

W_o_xavier = xavier_uniform((9, 9), np.random)
W_o_xavier


array([[-0.362, -0.530, 0.105, 0.205, -0.558, 0.014, -0.316, 0.168,
        -0.376],
       [0.220, -0.131, 0.504, -0.419, -0.184, -0.446, 0.490, 0.436,
        -0.280],
       [0.185, 0.366, 0.064, 0.034, -0.298, -0.470, 0.459, 0.462, 0.154],
       [-0.186, -0.174, 0.261, 0.459, 0.447, 0.323, 0.164, -0.480,
        -0.391],
       [0.460, 0.123, -0.567, -0.460, 0.189, -0.572, -0.392, 0.056,
        0.222],
       [0.175, -0.318, 0.245, -0.303, -0.202, 0.285, 0.173, 0.403, 0.182],
       [0.079, -0.469, -0.153, -0.271, -0.296, 0.546, -0.123, 0.453,
        0.151],
       [0.340, 0.003, 0.089, -0.009, -0.352, 0.257, -0.253, -0.549,
        0.168],
       [-0.373, 0.509, 0.524, 0.479, -0.150, -0.560, 0.495, -0.083,
        0.539]])

In [24]:
attentioin_final_xavier = multi_head_concat @ W_o_xavier
attentioin_final_xavier

array([[-0.527, 2.307, 1.301, 2.540, -1.043, -2.179, 0.652, -2.414,
        0.064],
       [-1.209, 1.367, 1.035, 2.530, -1.500, -1.542, -0.238, -2.086,
        -0.229],
       [-1.717, 2.076, 2.011, 3.351, -2.046, -1.744, 0.330, -2.720,
        0.073],
       [0.271, 4.543, 1.114, 2.881, -1.598, -2.683, 2.560, -1.092, 2.545],
       [0.971, 2.431, -0.914, 0.974, -0.450, -1.985, -0.195, -1.471,
        0.764],
       [-0.292, 0.379, -0.306, 1.443, -1.447, -0.807, -1.334, -1.573,
        -0.598],
       [-0.513, 1.345, 0.428, 2.203, -0.743, -0.642, -0.335, -2.470,
        -0.018],
       [0.703, 1.932, -1.898, 0.791, 0.982, -2.116, -1.202, -1.847,
        -1.269]])

In [26]:
attentioin_final_xavier.shape

(8, 9)

In [106]:
def layer_norm(x, eps=1e-5):
    # normalisasi per token (per baris)
    mu  = x.mean(axis=1, keepdims=True)
    var = x.var(axis=1, keepdims=True)
    return (x - mu) / np.sqrt(var + eps), mu.squeeze(), np.sqrt(var + eps).squeeze()

residual = V + attention_output
residual

array([[ 0.04405829,  0.10687363,  0.21846838],
       [ 1.78466909,  1.76979464, -0.57911326],
       [ 0.3621931 , -1.92369042,  2.37401561],
       [-0.35070644, -3.22650534,  4.33374449],
       [-0.37191269, -0.06044314,  1.90472948],
       [ 1.9923561 , -0.3408147 ,  1.84617628],
       [ 1.375     ,  3.125     , -1.625     ],
       [-0.38446667, -1.41985083,  2.28828779]])

In [64]:
normed, row_mean, row_std = layer_norm(residual)
normed

array([[-1.09531144, -0.2252231 ,  1.32053454],
       [ 0.71378865,  0.70039805, -1.4141867 ],
       [ 0.05203195, -1.24992963,  1.19789769],
       [-0.19349487, -1.11647895,  1.30997382],
       [-0.85597265, -0.54693334,  1.40290599],
       [ 0.77449179, -1.411994  ,  0.63750221],
       [ 0.21242937,  1.10463271, -1.31706208],
       [-0.34935844, -1.0121044 ,  1.36146284]])

In [65]:
row_mean

array([0.12313343, 0.99178349, 0.27083943, 0.25217757, 0.49079122,
       1.1659059 , 0.95833333, 0.16132343])

In [66]:
row_std

array([0.07219421, 1.11081285, 1.75572272, 3.11576221, 1.00786387,
       1.06708711, 1.96143627, 1.56226399])

## FFNN

In [ ]:
# import numpy as np
# def xavier_uniform(shape, rng=np.random):
#     fan_in, fan_out = shape[0], shape[1]
#     limit = np.sqrt(6.0 / (fan_in + fan_out))
#     return rng.uniform(-limit, limit, size=shape)

# # === FFNN layer 1 ===
# def ffnn_layer1(X, out_dim=18, bias_value=1.0, seed=42):
#     """
#     X         : numpy array (N, D_in) -> input hasil LayerNorm
#     out_dim   : jumlah neuron layer FF pertama (default 18)
#     bias_value: nilai bias (default 1)
#     seed      : agar hasil acak reproducible
#     """
#     np.random.seed(seed)

#     N, D_in = X.shape  # misal (8, 9)
#     D_out = out_dim    # misal 18

#     # Inisialisasi bobot dan bias
#     W1 = xavier_uniform((D_in, D_out))
#     b1 = np.full((D_out,), bias_value, dtype=float)

#     # Linear transform (belum ReLU)
#     Z = np.dot(X, W1) + b1  # (8×9) · (9×18) + (1×18) -> (8×18)

#     return Z, W1, b1

In [125]:
import numpy as np

# === He Initialization (Kaiming) ===
def he_initialization(shape, rng=np.random):
    """
    shape: (fan_in, fan_out)
    menggunakan distribusi normal std = sqrt(2/fan_in)
    """
    fan_in = shape[0]
    std = np.sqrt(2.0 / fan_in)
    return rng.normal(0.0, std, size=shape)

# === FFNN Layer 1 ===
def ffnn_layer1(X, out_dim=18, bias_value=1.0, seed=42):
    """
    X         : numpy array (N, D_in) hasil LayerNorm
    out_dim   : jumlah neuron hidden layer FF pertama
    bias_value: nilai bias (bias default = 1)
    seed      : untuk reproducibility
    """
    np.random.seed(seed)

    N, D_in = X.shape
    D_out = out_dim

    # Bobot dari He Initialization
    W1 = he_initialization((D_in, D_out))
    b1 = np.full((D_out,), bias_value, dtype=float)

    # Linear transform
    Z = np.dot(X, W1) + b1

    return Z, W1, b1


In [126]:
np.set_printoptions(formatter={'float': lambda x: f"{x:0.3f}"})

In [128]:
layer_norm_x = np.array([
    [-1.095, -0.225, 1.320],
    [ 0.713,  0.701, -1.415],
    [ 0.052, -1.249,  1.198],
    [-0.193, -1.116,  1.310],
    [-0.855, -0.547,  1.403],
    [ 0.774, -1.412,  0.638],
    [ 0.212,  1.104, -1.316],
    [-0.349, -1.012,  1.362]
], dtype=float)

# Jalankan layer FFNN pertama
Z, W1, b1 = ffnn_layer1(layer_norm_x, out_dim=18, bias_value=1.0, seed=42)

print("Z shape:", Z.shape)
print("W1 shape:", W1.shape)
print("W1:\n", np.round(W1, 4))
print("b1 shape:", b1.shape)
print("\nPreview output Z (belum ReLU):\n", np.round(Z, 4))

Z shape: (8, 18)
W1 shape: (3, 18)
W1:
 [[ 0.4056 -0.1129  0.5288  1.2435 -0.1912 -0.1912  1.2894  0.6266 -0.3833
   0.443  -0.3784 -0.3803  0.1976 -1.5622 -1.4084 -0.4591 -0.827   0.2566]
 [-0.7414 -1.1531  1.1967 -0.1843  0.0551 -1.1633 -0.4445  0.0906 -0.9398
   0.3068 -0.4904 -0.2382 -0.4913  1.5124 -0.011  -0.8636  0.6716 -0.9968]
 [ 0.1705 -1.6001 -1.0845  0.1607  0.603   0.1399 -0.0944 -0.2459 -1.2072
  -0.5878 -0.3761  0.8631  0.2806 -1.4395  0.2646 -0.3144 -0.5527  0.4994]]
b1 shape: (18,)

Preview output Z (belum ReLU):
 [[ 0.9478 -0.729  -1.2798 -0.108   1.9928  1.6558 -0.4366 -0.031   0.0377
  -0.3299  1.0282  2.6093  1.2646  0.4701  2.894   1.282   1.0249  1.6026]
 [ 0.5281  2.3752  3.7505  1.53    0.0492 -0.1498  1.7414  1.8581  1.7761
   2.3626  0.9186 -0.6594  0.3995  2.9833 -0.3863  0.5122  1.6632 -0.2225]
 [ 2.1514  0.5175 -1.7664  1.4875  1.6435  2.6106  1.5091  0.6249  0.7076
  -0.0642  1.1423  2.3117  1.96   -2.6947  1.2575  1.6781 -0.544   2.8567]
 [ 1.9725  0.212

In [131]:
Z1_relu = np.maximum(0, Z)
Z1_relu

array([[0.94782862, 0.        , 0.        , 0.        , 1.9928435 ,
        1.65577285, 0.        , 0.        , 0.0376763 , 0.        ,
        1.02820416, 2.60932008, 1.26455424, 0.47014783, 2.89395587,
        1.28200256, 1.02485451, 1.60257586],
       [0.52813873, 2.37524633, 3.75045444, 1.52998177, 0.04915356,
        0.        , 1.74138605, 1.85813744, 1.77610195, 2.36256066,
        0.9186274 , 0.        , 0.39946912, 2.98325388, 0.        ,
        0.51216375, 1.66324019, 0.        ],
       [2.15139881, 0.51752627, 0.        , 1.48747452, 1.64353342,
        2.61064938, 1.50909088, 0.62493582, 0.70761989, 0.        ,
        1.14227841, 2.3117344 , 1.9600108 , 0.        , 1.25753519,
        1.67811137, 0.        , 2.85668283],
       [1.97252936, 0.21260995, 0.        , 1.17628962, 1.76523802,
        2.51843857, 1.12349004, 0.45592721, 0.54133597, 0.        ,
        1.12763116, 2.46989485, 1.87768959, 0.        , 1.63076161,
        1.64051634, 0.        , 2.71718026],
    

In [124]:
Z1_relu.shape

(8, 18)

In [132]:
hidden_dim = Z1_relu.shape[1]                
D_out      = layer_norm_x.shape[1]       

# Inisialisasi W2, b2 dengan He Initialization & bias=1
W2 = he_initialization((hidden_dim, D_out))
b2 = np.full((D_out,), 1.0, dtype=float)

# Linear layer-2 (tanpa aktivasi)
Z2 = Z1_relu @ W2 + b2    # shape: (N, D_out)

print("W2 shape:", W2.shape)   # (hidden_dim, D_out)
print("b2 shape:", b2.shape)   # (D_out,)
print("Z2 shape:", Z2.shape)   # (N, D_out)
print("W2:\n", np.round(W2, 4))
print("\nPreview Z2 (tanpa residual & layer norm):\n", np.round(Z2, 4))


W2 shape: (18, 3)
b2 shape: (3,)
Z2 shape: (8, 3)
W2:
 [[ 0.3437  0.3104 -0.2797]
 [-0.1031  0.1104  0.3252]
 [-0.1597 -0.0619 -0.3688]
 [-0.3987  0.2708  0.4521]
 [-0.024   0.3345  0.1205]
 [-0.215   0.1205  0.5127]
 [-0.0119  0.5215 -0.8732]
 [ 0.274   0.029  -0.0997]
 [ 0.0306 -0.6625 -0.0732]
 [ 0.119   0.4926 -0.1728]
 [-0.2695 -0.1673  0.3051]
 [ 0.1096 -0.1766  0.1711]
 [ 0.0324  0.3229 -0.234 ]
 [-0.1092 -0.1307 -0.4878]
 [ 0.0987  0.087   0.0017]
 [-0.0782 -0.4718 -0.1402]
 [-0.1142 -0.2674 -0.0538]
 [ 0.1347  0.6287  0.0582]]

Preview Z2 (tanpa residual & layer norm):
 [[ 1.2056  2.23    1.9194]
 [-0.2403  1.4732 -2.7347]
 [ 1.0535  4.4759  1.562 ]
 [ 1.1572  4.1466  1.745 ]
 [ 1.2937  2.7945  2.3166]
 [ 0.7825  5.3057  0.864 ]
 [-0.3591  0.4988 -2.875 ]
 [ 1.2209  3.903   1.8559]]


In [133]:
Z2_out = layer_norm_x + Z2
Z2_out

array([[ 0.11057643,  2.00497583,  3.23942287],
       [ 0.47270993,  2.17423617, -4.14966024],
       [ 1.10546884,  3.22689362,  2.76001756],
       [ 0.96423579,  3.0306197 ,  3.05500759],
       [ 0.43869373,  2.24749984,  3.71956225],
       [ 1.55647262,  3.89367855,  1.50201042],
       [-0.14710173,  1.60279417, -4.1909685 ],
       [ 0.87190155,  2.89099214,  3.21791536]])

In [134]:
def layer_norm(x, eps=1e-5):
    # normalisasi per token (per baris)
    mu  = x.mean(axis=1, keepdims=True)
    var = x.var(axis=1, keepdims=True)
    return (x - mu) / np.sqrt(var + eps), mu.squeeze(), np.sqrt(var + eps).squeeze()

Z2_norm, Z_row_mean, Z_row_std = layer_norm(Z2_out)
Z2_norm

array([[-1.3012377 ,  0.17095618,  1.13028152],
       [ 0.36438525,  1.00119887, -1.36558412],
       [-1.3828494 ,  0.94789603,  0.43495338],
       [-1.4141332 ,  0.69462261,  0.71951058],
       [-1.26442527,  0.08365706,  1.18076821],
       [-0.68253711,  1.41392659, -0.73138947],
       [ 0.31515236,  1.03636967, -1.35152203],
       [-1.4024562 ,  0.54367328,  0.85878292]])

In [135]:
print("Z1 mean:", Z_row_mean)
print("Z1 std :", Z_row_std)

Z1 mean: [ 1.78499171 -0.50090471  2.36412667  2.34995436  2.13525194  2.3173872
 -0.91175869  2.32693635]
Z1 std : [1.28678664 2.67193757 0.91019154 0.97990668 1.34176234 1.11483253
 2.42630881 1.03749037]


In [136]:
N, D = Z2_norm.shape
M = D

W_linear = he_initialization((D, M))
b_linear = np.ones((M,), dtype=float)

y_linear = Z2_norm @ W_linear + b_linear

print("W_linear shape:", W_linear.shape)
print("Y_linear:", y_linear.shape)
print("Preview Y_linear:\n", np.round(y_linear, 3))

W_linear shape: (3, 3)
Y_linear: (8, 3)
Preview Y_linear:
 [[0.545 1.366 3.35 ]
 [1.269 0.691 2.481]
 [0.62  1.238 5.061]
 [0.575 1.297 4.592]
 [0.547 1.372 3.116]
 [0.941 0.931 4.934]
 [1.256 0.699 2.629]
 [0.558 1.323 4.266]]


In [137]:
W_linear

array([[ 0.21028901, -0.06078484, -1.56667014],
       [-0.02164849,  0.04917776,  2.01122876],
       [-0.15706207,  0.24621237, -0.02834204]])

## Classification

In [138]:
CLS = y_linear[0]       
M   = CLS.shape[0]

np.random.seed(77)

# Linear untuk [CLS] (He Initialization)
H_cls = 9
W_cls = he_initialization((M, H_cls))
b_cls = np.ones((H_cls,), dtype=float)

cls_linear = CLS @ W_cls + b_cls      
cls_tanh   = np.tanh(cls_linear)      

print("CLS shape:", CLS.shape)
print("W_cls shape:", W_cls.shape)
print("cls_linear:\n", np.round(cls_linear, 2))
print("cls_tanh:\n", np.round(cls_tanh, 2))

CLS shape: (3,)
W_cls shape: (3, 9)
cls_linear:
 [ 0.09  1.2   1.74 -1.9  -2.63 -1.67 -2.83  3.68 -0.63]
cls_tanh:
 [ 0.09  0.83  0.94 -0.96 -0.99 -0.93 -0.99  1.   -0.56]


In [139]:
CLS

array([0.54513871, 1.36579206, 3.35040773])

In [140]:
W_cls

array([[ 0.18325845,  0.54013819, -0.47337013,  0.33297655,  0.38087881,
        -1.61234323,  0.22498401, -1.28663954,  1.28059071],
       [-0.59179266,  0.24975791,  0.95035479, -0.8565828 , -0.7973726 ,
        -0.52178279,  1.38594516, -1.05010875, -0.15444979],
       [-0.06034944, -0.13104392, -0.0888295 , -0.56908506, -0.82124109,
        -0.32051169, -1.74550076,  1.43779405, -0.63212612]])

In [141]:
feat = cls_tanh                
F    = feat.shape[0]
num_labels = 2

np.random.seed(1313)

# Bobot classifier (He Initialization) dan bias=1 
W_clf = he_initialization((F, num_labels))
b_clf = np.ones((num_labels,), dtype=float)

logits = feat @ W_clf + b_clf      # (2,)
# Softmax
logits_shift = logits - logits.max()
probs = np.exp(logits_shift) / np.exp(logits_shift).sum()

print("W_clf shape:", W_clf.shape)
print("logits:", np.round(logits, 3))
print("softmax probs (Class A, Class B):", np.round(probs, 4))

W_clf shape: (9, 2)
logits: [0.809 1.11 ]
softmax probs (Class A, Class B): [0.4253 0.5747]


In [142]:
W_clf

array([[-0.0059573 , -0.68222747],
       [ 0.27895013, -0.16084336],
       [ 0.4778046 ,  0.22394529],
       [ 0.06251485, -0.3049946 ],
       [ 0.52724495, -0.22665598],
       [ 0.21025446,  0.68126781],
       [ 0.4707731 , -0.29790648],
       [ 0.38948355,  0.07968127],
       [ 0.02938697,  0.29220427]])